In [1]:
!pip install recommenders --user

In [2]:
!pip install pymanopt

In [3]:
import numpy as np
import sys
import time
import pandas as pd

from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.datasets import movielens
from recommenders.models.rlrmc.RLRMCdataset import RLRMCdataset 
from recommenders.models.rlrmc.RLRMCalgorithm import RLRMCalgorithm 
# Pymanopt installation is required via
# pip install pymanopt 
from recommenders.evaluation.python_evaluation import (
    rmse, mae
)

C:\Users\hp\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
rank_parameter = 10
regularization_parameter = 0.001
initialization_flag = 'svd'
maximum_iteration = 100
maximum_time = 300
verbosity=0
compute_iter_rmse=True

In [5]:
df = pd.read_csv('ratings.csv', sep=',', usecols=['userID', 'itemID', 'rating','timestamp'])
df.head()

,userID,itemID,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [18]:
train, test = python_random_split(df,[0.8, 0.2])

In [19]:
data = RLRMCdataset(train=train, test=test)

In [20]:
model = RLRMCalgorithm(rank = rank_parameter,
                       C = regularization_parameter,
                       model_param = data.model_param,
                       initialize_flag = initialization_flag,
                       maxiter=maximum_iteration,
                       max_time=maximum_time)

In [21]:
start_time = time.time()

model.fit(data,verbosity=verbosity)

# fit_and_evaluate will compute RMSE on the validation set (if given) at every iteration
# model.fit_and_evaluate(data,verbosity=verbosity)

train_time = time.time() - start_time # train_time includes both model initialization and model training time. 

print("Took {} seconds for training.".format(train_time))

Took 1.6222953796386719 seconds for training.


In [22]:
predictions_ndarr = model.predict(test['userID'].values,test['itemID'].values)

In [23]:
predictions_df = pd.DataFrame(data={"userID": test['userID'].values, "itemID":test['itemID'].values, "prediction":predictions_ndarr})

## Compute test RMSE 
eval_rmse = rmse(test, predictions_df)
## Compute test MAE 
eval_mae = mae(test, predictions_df)

print("RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae, sep='\n')

RMSE:	1.011215
MAE:	0.765671
